In [234]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import warnings
warnings.filterwarnings("ignore")

In [130]:
df = pd.read_csv("C:/Users/fadhe/Downloads/baseline_new.csv")
df.head()

,Unnamed: 0,Timestamp,% Baseline
0,0,2014-01-01 07:00:00,0.0079
1,1,2014-01-01 08:00:00,0.1019
2,2,2014-01-01 09:00:00,0.3932
3,3,2014-01-01 10:00:00,0.5447
4,4,2014-01-01 11:00:00,0.5485


In [131]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:00:00')

df = df.rename(columns={'Timestamp': 'date_time'})

df.set_index('date_time', inplace=True)

df.drop(columns="Unnamed: 0", inplace = True)

df.head(15)

,% Baseline
date_time,
2014-01-01 07:00:00,0.0079
2014-01-01 08:00:00,0.1019
2014-01-01 09:00:00,0.3932
2014-01-01 10:00:00,0.5447
2014-01-01 11:00:00,0.5485
2014-01-01 12:00:00,0.3548
2014-01-01 13:00:00,0.1934
2014-01-01 14:00:00,0.1348
2014-01-01 15:00:00,0.0452


In [132]:
df_si = pd.read_excel("C:/Users/fadhe/Downloads/Solar_irradiance_full.xlsx")
df_si.drop(columns="Cloud Type", inplace = True)

df_si['date_time'] = df_si['Year'].astype(str) + '-' + \
                 df_si['Month'].astype(str).str.zfill(2) + '-' + \
                 df_si['Day'].astype(str).str.zfill(2) + '-' + \
                 df_si['Hour'].astype(str).str.zfill(2) + '-' + \
                 df_si['Minute'].astype(str).str.zfill(2)

df_si.drop(columns=["Year", "Month", "Day", "Hour", "Minute"], inplace = True)

df_si['date_time'] = pd.to_datetime(df_si['date_time'], format='%Y-%m-%d-%H-%M')

df_si['date_time'] = df_si['date_time'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_si.set_index("date_time", inplace = True)

df_si.head(10)

,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
date_time,,,,,,,,,,,,,
2014-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-6,124.02,0.12,3.5,78.43,-3,1010
2014-01-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-5,135.09,0.12,3.8,81.74,-3,1010
2014-01-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-5,145.77,0.12,4.2,81.25,-3,1010
2014-01-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-6,155.07,0.12,4.5,74.14,-3,1010
2014-01-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-8,160.55,0.12,4.6,72.80,-4,1010
2014-01-01 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-8,158.86,0.12,4.6,68.48,-4,1010
2014-01-01 06:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-9,151.26,0.12,4.5,71.57,-5,1010
2014-01-01 07:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-9,141.17,0.12,4.4,69.95,-5,1020
2014-01-01 08:00:00,0.0,0.0,0.0,0.0,0.0,0.0,-9,130.24,0.12,4.3,74.24,-6,1020


In [133]:
df_w = pd.read_csv("D:/DAC/dataset/Weather.csv")

df_w['date_time'] = pd.to_datetime(df_w['date_time'], format='%m/%d/%Y %H:%M')

df_w['date_time'] = df_w['date_time'].dt.strftime('%Y-%m-%d %H:%M:00')

df_w.set_index("date_time", inplace = True)

df_w.drop(columns=["moonrise", "moonset", "sunrise", "sunset"], inplace = True)

df_w.head()

,maxtempC,mintempC,totalSnow_cm,sunHour,uvIndex,moon_illumination,DewPointC,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
date_time,,,,,,,,,,,,,,,,,,,
2014-01-01 00:00:00,-3,-6,0.0,8.7,2,1,-12,-11,-5,-11,28,3,58,0.0,1022,-4,10,273,20
2014-01-01 01:00:00,-3,-6,0.0,8.7,2,1,-12,-12,-5,-12,31,4,58,0.0,1023,-5,10,274,22
2014-01-01 02:00:00,-3,-6,0.0,8.7,2,1,-12,-12,-5,-12,34,5,58,0.0,1024,-5,10,276,25
2014-01-01 03:00:00,-3,-6,0.0,8.7,2,1,-12,-13,-5,-13,36,6,58,0.0,1024,-5,10,278,27
2014-01-01 04:00:00,-3,-6,0.0,8.7,2,1,-13,-13,-6,-13,32,9,57,0.0,1025,-6,10,271,23


In [134]:
print(len(df_si))
print(len(df_w))
print(len(df))

35064
35064
23268


In [135]:
# df_siy = pd.concat([df_si.head(len(df)), df], axis = 1)

# df_siy.tail()

# fig, ax = plt.subplots(figsize=(15, 5))

# sns.heatmap(df_siy.corr(), annot=True, cmap='coolwarm', fmt='.2f', ax=ax)

# plt.show()

In [136]:
# df_wy = pd.concat([df_w.head(len(df)), df], axis = 1)

# df_wy.dropna(inplace = True)

# fig, ax = plt.subplots(figsize=(15, 5))

# sns.heatmap(df_wy.corr(), annot=True, cmap='coolwarm', fmt='.2f', ax=ax)

# plt.show()

In [137]:
var_x = pd.concat([df_si, df_w], axis = 1)
var_x.dropna(inplace = True)
var_x.tail(7)

,DHI,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,...,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,pressure,tempC,visibility,winddirDegree,windspeedKmph
date_time,,,,,,,,,,,,,,,,,,,,,
2017-12-31 17:00:00,93.000000,659.000000,362.000000,84.000000,895.000000,449.000000,-14,65.93,0.866,4.8,...,-20,32,19,65,0.0,1022,-11,10,308,27
2017-12-31 18:00:00,81.280832,73.844477,126.112432,52.572065,278.575532,227.870728,-14,69.40,0.866,4.8,...,-21,33,7,67,0.0,1022,-12,10,310,27
2017-12-31 19:00:00,81.280832,73.844477,126.112432,52.572065,278.575532,227.870728,-13,75.29,0.866,4.8,...,-22,33,4,68,0.0,1022,-12,10,309,27
2017-12-31 20:00:00,81.280832,73.844477,126.112432,52.572065,278.575532,227.870728,-13,83.05,0.866,4.8,...,-22,34,2,69,0.0,1022,-12,10,308,27
2017-12-31 21:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13,92.18,0.866,4.8,...,-23,34,0,70,0.0,1023,-13,10,308,28
2017-12-31 22:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13,102.27,0.866,4.9,...,-23,33,0,71,0.0,1023,-13,10,309,27
2017-12-31 23:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-14,112.97,0.866,4.9,...,-23,32,0,72,0.0,1023,-13,10,310,26


In [138]:
var_x.columns

Index(['DHI', 'DNI', 'GHI', 'Clearsky DHI', 'Clearsky DNI', 'Clearsky GHI',
       'Dew Point', 'Solar Zenith Angle', 'Surface Albedo', 'Wind Speed',
       'Relative Humidity', 'Temperature', 'Pressure', 'maxtempC', 'mintempC',
       'totalSnow_cm', 'sunHour', 'uvIndex', 'moon_illumination', 'DewPointC',
       'FeelsLikeC', 'HeatIndexC', 'WindChillC', 'WindGustKmph', 'cloudcover',
       'humidity', 'precipMM', 'pressure', 'tempC', 'visibility',
       'winddirDegree', 'windspeedKmph'],
      dtype='object')

In [139]:
var_x.drop(columns=["Pressure", "DHI", "winddirDegree", "windspeedKmph", "totalSnow_cm", "moon_illumination", "pressure"], inplace = True)

In [140]:
df = df.reset_index()
var_x = var_x.reset_index()

In [141]:
df['date_time'] = pd.to_datetime(df['date_time'])
var_x['date_time'] = pd.to_datetime(var_x['date_time'])

df_full = pd.merge_asof(df.reset_index(), var_x.reset_index(), on='date_time', direction='backward')

# Set index kembali
df_full.set_index('date_time', inplace=True)

In [142]:
data_na = df_full[df_full["% Baseline"].isna()]
data_na.drop(columns=["index_x", "index_y"], inplace = True)
temp = pd.DataFrame(data_na["% Baseline"])
data_na.drop(columns="% Baseline", inplace = True)
data_na["% Baseline"] = temp
data_na.head(10)

,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,Relative Humidity,...,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,tempC,visibility,% Baseline
date_time,,,,,,,,,,,,,,,,,,,,,
2014-01-01 17:00:00,307.0,288.0,72.0,854.0,421.0,-10,65.84,0.120,3.5,53.34,...,-8,-3,-8,13,58,49,0.0,-3,10,NaN
2014-01-01 18:00:00,495.0,281.0,66.0,811.0,353.0,-10,69.29,0.120,3.4,54.33,...,-8,-3,-8,8,64,49,0.0,-3,10,NaN
2014-01-01 19:00:00,427.0,187.0,56.0,703.0,236.0,-9,75.16,0.120,3.2,55.62,...,-6,-3,-6,6,76,49,0.0,-3,10,NaN
2014-01-01 20:00:00,468.0,94.0,36.0,468.0,94.0,-9,82.92,0.120,3.0,57.40,...,-5,-3,-5,5,88,49,0.0,-4,10,NaN
2014-01-01 21:00:00,0.0,0.0,0.0,0.0,0.0,-8,92.05,0.120,2.8,65.28,...,-3,-3,-3,3,100,49,0.0,-4,10,NaN
2014-01-01 22:00:00,0.0,0.0,0.0,0.0,0.0,-8,102.14,0.120,2.7,71.26,...,-5,-4,-5,8,100,52,0.0,-4,10,NaN
2014-01-01 23:00:00,0.0,0.0,0.0,0.0,0.0,-8,112.84,0.120,2.4,71.33,...,-7,-4,-7,12,100,55,0.0,-4,10,NaN
2014-01-02 07:00:00,0.0,0.0,0.0,0.0,0.0,-7,141.20,0.866,1.9,77.72,...,-10,-3,-10,37,100,86,0.0,-3,7,NaN
2014-01-02 08:00:00,0.0,0.0,0.0,0.0,0.0,-6,130.28,0.866,2.4,80.11,...,-10,-2,-10,43,100,86,0.5,-2,3,NaN


In [143]:
data_na.isna().sum()

DNI                      0
GHI                      0
Clearsky DHI             0
Clearsky DNI             0
Clearsky GHI             0
Dew Point                0
Solar Zenith Angle       0
Surface Albedo           0
Wind Speed               0
Relative Humidity        0
Temperature              0
maxtempC                 0
mintempC                 0
sunHour                  0
uvIndex                  0
DewPointC                0
FeelsLikeC               0
HeatIndexC               0
WindChillC               0
WindGustKmph             0
cloudcover               0
humidity                 0
precipMM                 0
tempC                    0
visibility               0
% Baseline            6447
dtype: int64

In [144]:
data_train = df_full.dropna()
data_train.drop(columns=["index_x", "index_y"], inplace = True)
temp = pd.DataFrame(data_train["% Baseline"])
data_train.drop(columns="% Baseline", inplace = True)
data_train["% Baseline"] = temp
data_train.head(10)

,DNI,GHI,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,Solar Zenith Angle,Surface Albedo,Wind Speed,Relative Humidity,...,FeelsLikeC,HeatIndexC,WindChillC,WindGustKmph,cloudcover,humidity,precipMM,tempC,visibility,% Baseline
date_time,,,,,,,,,,,,,,,,,,,,,
2014-01-01 07:00:00,0.0,0.0,0.0,0.0,0.0,-9,141.17,0.12,4.4,69.95,...,-13,-5,-13,24,13,52,0.0,-5,10,0.0079
2014-01-01 08:00:00,0.0,0.0,0.0,0.0,0.0,-9,130.24,0.12,4.3,74.24,...,-12,-5,-12,26,11,49,0.0,-5,10,0.1019
2014-01-01 09:00:00,0.0,0.0,0.0,0.0,0.0,-10,119.14,0.12,4.3,73.07,...,-11,-4,-11,27,9,46,0.0,-4,10,0.3932
2014-01-01 10:00:00,0.0,0.0,0.0,0.0,0.0,-10,108.20,0.12,4.1,72.43,...,-10,-4,-10,25,15,46,0.0,-4,10,0.5447
2014-01-01 11:00:00,0.0,0.0,0.0,0.0,0.0,-10,97.73,0.12,3.9,72.47,...,-10,-3,-10,23,21,45,0.0,-3,10,0.5485
2014-01-01 12:00:00,232.0,23.0,15.0,232.0,23.0,-9,88.00,0.12,3.7,68.04,...,-9,-3,-9,22,28,44,0.0,-3,10,0.3548
2014-01-01 13:00:00,630.0,162.0,47.0,630.0,162.0,-9,79.40,0.12,3.7,64.00,...,-9,-3,-9,22,34,45,0.0,-3,10,0.1934
2014-01-01 14:00:00,781.0,298.0,62.0,781.0,298.0,-10,72.39,0.12,3.6,58.17,...,-9,-3,-9,23,39,46,0.0,-3,10,0.1348
2014-01-01 15:00:00,847.0,394.0,69.0,847.0,394.0,-10,67.49,0.12,3.6,52.61,...,-8,-3,-8,24,45,47,0.0,-3,10,0.0452


In [145]:
data_train.isna().sum()

DNI                   0
GHI                   0
Clearsky DHI          0
Clearsky DNI          0
Clearsky GHI          0
Dew Point             0
Solar Zenith Angle    0
Surface Albedo        0
Wind Speed            0
Relative Humidity     0
Temperature           0
maxtempC              0
mintempC              0
sunHour               0
uvIndex               0
DewPointC             0
FeelsLikeC            0
HeatIndexC            0
WindChillC            0
WindGustKmph          0
cloudcover            0
humidity              0
precipMM              0
tempC                 0
visibility            0
% Baseline            0
dtype: int64

In [167]:
X_train = data_train.iloc[:, :-1].values
y_train = data_train.iloc[:, -1].values

X_test = data_na.iloc[:, :-1].values

In [169]:
ct = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
    ],
    remainder="passthrough"
)

sc = StandardScaler()

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)

y_train = sc.fit_transform(y_train)
# y_test = sc.transform(y_test)

y_train = y_train.reshape(-1)
# y_test = y_test.reshape(-1)

In [171]:
# Definisikan fungsi loss RMSE
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# Buat model ANN
ann = Sequential()

ann.add(Dense(units=200, activation="relu"))
ann.add(Dense(units=200, activation="relu"))
ann.add(Dense(units=200, activation="relu"))
ann.add(Dense(units=1))

# Kompilasi model dengan fungsi loss RMSE
ann.compile(optimizer="adam", loss=rmse)

# fit model
ann.fit(X_train, y_train, batch_size = 32, epochs = 500)

Epoch 1/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.5940
Epoch 2/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4760
Epoch 3/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4457
Epoch 4/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4349
Epoch 5/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4220
Epoch 6/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.4170
Epoch 7/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3953
Epoch 8/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3927
Epoch 9/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3846
Epoch 10/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3736
Epoch 11/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3588
Epoch 12/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3568
Epoch 13/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3451
Epoch 14/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3422
Epoch 15/500
526/526 ━━━━━━━━━━━━━━━━━━━━ 1

In [173]:
import os
os.chdir("D:/DAC")

In [175]:
# ann.save('isi_kosong.h5')

In [179]:
y_pred = ann.predict(X_test)

202/202 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [183]:
y_pred = sc.inverse_transform(y_pred)

y_pred = y_pred.reshape(-1)

y_pred = np.around(y_pred, decimals=2)

print(y_pred)

[0.11 0.14 0.24 ... 0.24 0.2  0.24]


In [185]:
df_full['% Baseline'].loc[df_full['% Baseline'].isna()] = y_pred

In [189]:
df_full.drop(columns=["index_x", "index_y"], inplace = True)
df_full.isna().sum()

% Baseline            0
DNI                   0
GHI                   0
Clearsky DHI          0
Clearsky DNI          0
Clearsky GHI          0
Dew Point             0
Solar Zenith Angle    0
Surface Albedo        0
Wind Speed            0
Relative Humidity     0
Temperature           0
maxtempC              0
mintempC              0
sunHour               0
uvIndex               0
DewPointC             0
FeelsLikeC            0
HeatIndexC            0
WindChillC            0
WindGustKmph          0
cloudcover            0
humidity              0
precipMM              0
tempC                 0
visibility            0
dtype: int64

In [207]:
len(df_full)

23268

In [191]:
temp = df_full["% Baseline"]
df_full.drop(columns="% Baseline", inplace = True)
df_full["% Baseline"] = temp

In [211]:
df_test = pd.concat([df_si, df_w], axis = 1)
df_test.dropna(inplace = True)
df_test = df_test.tail(2208)
df_test.drop(columns=["Pressure", "DHI", "winddirDegree", "windspeedKmph", "totalSnow_cm", "moon_illumination", "pressure"], inplace = True)
len(df_test)

2208

In [213]:
df_last = pd.read_csv("D:/DAC/dataset/analysis.csv")
df_last = df_last.rename(columns={'Timestamp': 'date_time'})
df_last.set_index("date_time", inplace = True)
df_last = pd.DataFrame(df_last["% Baseline"])
df_last = df_last.tail(2208)
len(df_last)

2208

In [219]:
X_train = df_full.iloc[:, :-1].values
y_train = df_full.iloc[:, -1].values

X_test = df_test

In [221]:
ct = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24])
    ],
    remainder="passthrough"
)

sc = StandardScaler()

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)

y_train = sc.fit_transform(y_train)
# y_test = sc.transform(y_test)

y_train = y_train.reshape(-1)
# y_test = y_test.reshape(-1)

In [ ]:
# Definisikan fungsi loss RMSE
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

# Buat model ANN
ann2 = Sequential()

ann2.add(Dense(units=250, activation="relu"))
ann2.add(Dense(units=250, activation="relu"))
ann2.add(Dense(units=250, activation="relu"))
ann2.add(Dense(units=1))

# Kompilasi model dengan fungsi loss RMSE
ann2.compile(optimizer="adam", loss=rmse)

# fit model
ann2.fit(X_train, y_train, batch_size = 32, epochs = 500)

Epoch 1/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.5676
Epoch 2/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4591
Epoch 3/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4406
Epoch 4/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4168
Epoch 5/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.4052
Epoch 6/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3912
Epoch 7/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3865
Epoch 8/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3828
Epoch 9/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3666
Epoch 10/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3533
Epoch 11/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3471
Epoch 12/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3405
Epoch 13/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.3312
Epoch 14/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.3208
Epoch 15/500
728/728 ━━━━━━━━━━━━━━━━━━━━ 2

In [226]:
# ann2.save('isi_test.h5')

In [228]:
ann2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 250)            │         6,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 250)            │        62,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 250)            │        62,750 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           251 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 396,755 (1.51 MB)

 Trainable params: 132,251 (516.61 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 264,504 (1.01 MB)

In [230]:
y_banding = ann2.predict(X_train)
y_banding = sc.inverse_transform(y_banding)
y_banding = y_banding.reshape(-1)
y_banding = np.around(y_banding, decimals=2)

y_train = df_full.iloc[:, -1].values
selisih = y_train - y_banding

728/728 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [250]:
n_input = 5
n_features = 1

generator = TimeseriesGenerator(selisih, selisih, length = n_input, batch_size = 1)

In [252]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))

rnn = Sequential()

rnn.add(LSTM(units = 200, activation = "relu", return_sequences = True, input_shape = (n_input, n_features)))
rnn.add(LSTM(units = 200, activation = "relu"))
rnn.add(Dense(units = 1))

rnn.compile(optimizer = "adam", loss = rmse)

In [256]:
rnn.fit(generator, epochs=8)

Epoch 1/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 279s 12ms/step - loss: 0.0124
Epoch 2/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 286s 12ms/step - loss: 0.0123
Epoch 3/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 304s 13ms/step - loss: 0.0124
Epoch 4/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 244s 10ms/step - loss: 0.0123
Epoch 5/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 287s 12ms/step - loss: 0.0122
Epoch 6/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 303s 13ms/step - loss: 0.0123
Epoch 7/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 287s 12ms/step - loss: 0.0122
Epoch 8/8
23263/23263 ━━━━━━━━━━━━━━━━━━━━ 292s 13ms/step - loss: 0.0123


In [258]:
# rnn.save('forecast_lstm.h5')

In [ ]:
test_prediction = []

first_eval_batch = selisih[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(2208):
    current_pred = rnn.predict(current_batch)[0]
    test_prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:], [[current_pred]], axis = 1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 599ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━

In [263]:
hasil_akhir = []

for i in range(len(test_prediction)):
    hasil_akhir.append(test_prediction[i][0])

hasil_akhir = np.array(hasil_akhir)

In [265]:
hasil_awal = ann2.predict(X_test)
hasil_awal = sc.inverse_transform(hasil_awal)
hasil_awal = hasil_awal.reshape(-1)
hasil_awal = np.around(hasil_awal, decimals=2)

final =  hasil_awal + hasil_akhir

69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [267]:
df_last["% Baseline"] = final
df_last = df_last.rename(columns={'% Baseline': 'nilai'})

In [269]:
df_test = pd.read_csv("D:/DAC/dataset/test.csv")

df_test = df_test.rename(columns={'Timestamp': 'date_time'})

df_test['date_time'] = pd.to_datetime(df_test['date_time'], format='%b %d, %Y %I%p')

df_test['date_time'] = df_test['date_time'].dt.strftime('%Y-%m-%d %H:00:00')

df_test.set_index('date_time', inplace=True)

In [271]:
df_test = df_test.reset_index()
df_last = df_last.reset_index()

In [273]:
df_test['date_time'] = pd.to_datetime(df_test['date_time'])
df_last['date_time'] = pd.to_datetime(df_last['date_time'])

df_test = pd.merge_asof(df_test.reset_index(), df_last.reset_index(), on='date_time', direction='backward')

df_test.set_index('date_time', inplace=True)
df_test.drop(columns=["index_x", "index_y"], inplace=True)

In [293]:
misal = df_test["nilai"]
misal = np.array(misal)
misal

array([0.03275439, 0.06275439, 0.1827544 , ..., 0.14275439, 0.12275439,
       0.13275439], dtype=float32)

In [295]:
dikumpul = pd.read_csv("C:/Users/fadhe/Downloads/sample_submission.csv")
dikumpul["% Baseline"] = misal

In [297]:
dikumpul

,Timestamp,% Baseline
0,"Oct 1, 2017 6am",0.032754
1,"Oct 1, 2017 7am",0.062754
2,"Oct 1, 2017 8am",0.182754
3,"Oct 1, 2017 9am",0.272754
4,"Oct 1, 2017 10am",0.522754
...,...,...
1072,"Dec 31, 2017 12pm",0.232754
1073,"Dec 31, 2017 1pm",0.132754
1074,"Dec 31, 2017 2pm",0.142754
1075,"Dec 31, 2017 3pm",0.122754


In [299]:
dikumpul.to_csv("yaa.csv", index=False)